In [3]:
import subprocess
import time
import os

import pyautogui
pyautogui.PAUSE = 0.3
pyautogui.FAILSAFE = True


import configparser
config = configparser.ConfigParser()
config.read('config.cfg')

['config.cfg']

In [4]:
def close_excel():
    try:
        os.system('TASKKILL /F /IM excel.exe')
        time.sleep(10)
    except Exception:
        print("KU")

def check_server_location():
    sap_dev = pyautogui.locateOnScreen('locations/sap_dev.png')
    if sap_dev is None:
        sap_dev = pyautogui.locateOnScreen('locations/sap_dev_default.png')
        if sap_dev is None:
            raise ValueError("cannot detect sap server location")
    return sap_dev

def remove_export_file_if_exists():
    if os.path.isfile(os.path.join(config['PATH']['LOCAL'], config['FILE']['EXPORT'])):
        os.remove(os.path.join(config['PATH']['LOCAL'], config['FILE']['EXPORT']))
        print(f'-- Found {config['FILE']['EXPORT']} file exists remove it first.')
        
def open_sap_gui(config):
    process = subprocess.Popen(config['PATH']['SAP_LOGON'])
    print('-- Open SAP GUI')
    time.sleep(5)
    
    return sap_gui

def select_sap_server():
    
    sap_dev = check_server_location()
    while sap_dev is None:
        time.sleep(3)
        sap_dev = check_server_location()
    pyautogui.moveTo(sap_dev)
    pyautogui.doubleClick()
    print('-- Open SAP server')
    time.sleep(3)
    
    
def fill_username_password():
    pyautogui.typewrite(config['CREDENTIALS']['SAP_USERNAME'], interval=0.1)
    pyautogui.press('tab')
    pyautogui.typewrite(config['CREDENTIALS']['SAP_PASSWORD'], interval=0.1)
    pyautogui.press('enter')
    print('-- SAP LOGGED IN')
    time.sleep(3)
    
def open_zfiaraging_module():
    pyautogui.typewrite('ZFIARAGING')
    pyautogui.press('enter')
    print('-- Open ZFIARAGING module')
    time.sleep(3)

def fill_zfiaraging_form(comp_code, profit_center, aging_date):
    pyautogui.typewrite(comp_code)
    for i in range(0,3):
        pyautogui.press('tab')

    if str(profit_center) == "nan":
        pass
    else:
        pyautogui.typewrite(profit_center)

    for i in range(0,18):
        pyautogui.press('tab')
    pyautogui.typewrite(aging_date)
    pyautogui.press('tab')
    pyautogui.press('down') # select Display ALV Grid Standard
    pyautogui.press('f8')
    time.sleep(10)
    
def check_report_load():
    check_report_load = pyautogui.locateOnScreen('locations/report_load_success.png')
    retry = 0
    while not check_report_load:
        time.sleep(30)
        check_report_load = pyautogui.locateOnScreen('locations/report_load_success.png')
        retry += 1
        if retry == 6:
            print('-- Report load failed')
            break
    print('-- Report load success')
    
def select_report_layout():
     # to select report layout
    pyautogui.hotkey('ctrl', 'f9')
    time.sleep(10)
    report_layout = pyautogui.locateOnScreen('locations/report_layout.png')
    pyautogui.moveTo(report_layout)
    pyautogui.click()
    time.sleep(10) # wait for new layout to reload
    
def export_report():
    
    # move to center of screen
    pyautogui.moveTo(pyautogui.size().width / 2,
                    pyautogui.size().height / 2)
    pyautogui.click(button='right')
    time.sleep(10)
    pyautogui.moveRel(10,150) # move to spreadsheet buttong
    pyautogui.click()
    time.sleep(10)
    pyautogui.press('enter') # alredy set default export to XLSX
    time.sleep(10)
    pyautogui.press('enter') # to save report to export.xlsx
    time.sleep(10)
    
    excel_loaded = pyautogui.locateOnScreen('locations/excel_loaded.png')
    retry = 0
    while not excel_loaded:
        pyautogui.press('enter') # to save report to export.xlsx
        time.sleep(10)
        excel_loaded = pyautogui.locateOnScreen('locations/excel_loaded.png')
        retry += 1
        if retry == 3:
            print('retry exceed')
            break
            
def rename_export_file(config, export_aging_date, key, profit_center):
    
    if os.path.isfile(os.path.join(config['PATH']['LOCAL'], config['FILE']['EXPORT'])): # check if export file exists

        if not os.path.exists(os.path.join(config['PATH']['LOCAL'], export_aging_date)): # check export_aging_date folder exists
            print(f"{export_aging_date} folder doesn't exists ; create a new folder")
            os.mkdir( 
                os.path.join(config['PATH']['LOCAL'], export_aging_date) 
            )

        if os.path.exists(os.path.join(config['PATH']['LOCAL'], '{}_{}.xlsx'.format(key, profit_center))): # if target file exists
            # remove the exist file and replace with current one
            print(f"{export_aging_date}\\{key}.xlsx is exists ; remove the exist file and replace with current one")
            os.remove(os.path.join(config['PATH']['LOCAL'], f"{export_aging_date}\\{key}.xlsx"))

        # move export file to target folder and rename
        os.rename(
            os.path.join(config['PATH']['LOCAL'], f"export.XLSX"),
            os.path.join(config['PATH']['LOCAL'], '{}_{}.xlsx'.format(key, profit_center))
                 )
        print(f'''
            Rename export file to {os.path.join(config['PATH']['LOCAL'], export_aging_date, '{}_{}.xlsx'.format(key, profit_center))}
        ''')

In [37]:
def extract_zfiaraging_report(key, comp_code, profit_center, aging_date, export_aging_date, config):
    
    remove_export_file_if_exists()
    sap_gui = open_sap_gui(config)
    select_sap_server()
    fill_username_password()
    open_zfiaraging_module()
    fill_zfiaraging_form(comp_code, profit_center, aging_date)
    check_report_load()
    select_report_layout() 
    export_report()
    rename_export_file(config, export_aging_date, key, profit_center)
    close_excel()
    sap_gui.terminate() # close SAP GUI

In [44]:
import pandas as pd
import numpy as np

_input = pd.read_csv('../CONFIG_compcode_subcompname.csv', dtype={'profit_center':str})
_input = _input[['sub_comp_name', 'comp_code', 'profit_center']]

aging_date = "31.03.2021"
export_aging_date = '310321'

for _tup in _input.itertuples():
    _, key, comp_code, profit_center = _tup
    key = str(key)
    comp_code = str(comp_code)
    profit_center = str(profit_center)
    print(f'-- {datetime.datetime.now()} - extract report for {key}')
    extract_zfiaraging_report(key, comp_code, profit_center, aging_date, export_aging_date, config)

-- 2021-04-16 19:20:46.094123 - extract report for PNG
open sap gui
open sap server
finish sap logon
enter ZFIARAGING module
number of retry : 1
report load success

            rename export file to D:\Users\pathairush.s.ASSETWORLDCORP-\Documents\SAP\310321\PNG_2472501000.xlsx
        
-- 2021-04-16 19:25:44.142667 - extract report for PNG
open sap gui
open sap server
finish sap logon
enter ZFIARAGING module
report load success
310321\PNG.xlsx is exists ; remove the exist file and replace with current one

            rename export file to D:\Users\pathairush.s.ASSETWORLDCORP-\Documents\SAP\310321\PNG_2478011000.xlsx
        
-- 2021-04-16 19:27:36.743154 - extract report for PTN
open sap gui
open sap server
finish sap logon
enter ZFIARAGING module
number of retry : 1
report load success

            rename export file to D:\Users\pathairush.s.ASSETWORLDCORP-\Documents\SAP\310321\PTN_2472502000.xlsx
        
-- 2021-04-16 19:32:48.208398 - extract report for PCM
open sap gui
open sap 